In [74]:
# The magic commands below allow reflecting the changes in an imported module without restarting the kernel.
%load_ext autoreload
%autoreload 2

# We need to add balsam and the modules it depends on to the Python search paths. 
import sys
sys.path.insert(0,'/soft/datascience/Balsam/0.3.5.1/env/lib/python3.6/site-packages/')
sys.path.insert(0,'/soft/datascience/Balsam/0.3.5.1/')

# We also need postgresql to be in the path
import os
os.environ['PATH'] ='/soft/datascience/Balsam/0.3.5.1/env/bin/:' + os.environ['PATH']
os.environ['PATH'] +=':/soft/datascience/PostgreSQL/9.6.12/bin/'

try:
    import balsam
except:
    print('Cannot find balsam, make sure balsam is installed or it is available in Python search paths')    

# We also need to activate Balsam database by setting the BALSAM_DB_PATH environment variable. 
# This is equivalent to `source balsamactivate jupyter_test` 
os.environ["BALSAM_DB_PATH"]='/lus/theta-fs0/projects/connectomics_aesp/balsam_database/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
def get_database_paths(verbose=True):
    """
    Prints the paths for existing balsam databases
    """
    try:
        from balsam.django_config.db_index import refresh_db_index
        databasepaths = refresh_db_index()
    except:
        databasepaths = None
    if verbose:
        if len(databasepaths) > 0:
            print(f'Found {len(databasepaths)} balsam database locations')
            for db in databasepaths:
                print(db)
        else:
            print('No balsam database found')
    return databasepaths

def get_active_database(verbose=True):
    """
    Gets the activate database set in environment variable BALSAM_DB_PATH
    Parameters:
    verbose: Boolean, (True): Prints verbose info (False): No print
    Returns
    -------
    str, path for the active database
    """
    try:
        db = os.environ["BALSAM_DB_PATH"]
        if verbose: print(f'Active balsam database path: {db}')
    except:
        if verbose: print('BALSAM_DB_PATH is not set')
        db = None
    return db
    
def add_app(name, executable, description='', envscript='', preprocess='', postprocess='', checkexe=False):
    """
    Adds a new app to the balsam database.
    """
    from balsam.core.models import ApplicationDefinition as App
    import shutil
    newapp = App()
    if checkexe:
        if shutil.which(executable):        
            print('{} is found'.format(executable))
        else:
            print('{} is not found'.format(executable))
            return newapp
        
    if App.objects.filter(name=name).exists():
        print("An application named {} already exists".format(name))
    else:
        newapp.name        = name
        newapp.executable  = executable
        newapp.description = description
        newapp.envscript   = envscript
        newapp.preprocess  = preprocess
        newapp.postprocess = postprocess
        newapp.save()
    return newapp

def get_apps(verbose=True):
    """
    Returns all apps as a list
    """
    try:
        from balsam.core.models import ApplicationDefinition as App
        apps = App.objects.all()
    except:
        apps = None
    return apps
        
def get_job():
    from balsam.launcher.dag import BalsamJob
    return BalsamJob()

def add_job(name, workflow, application, description='', args='', num_nodes=1, ranks_per_node=1,cpu_affinity='depth',data={},environ_vars={}):
    from balsam.launcher.dag import BalsamJob
    job                = BalsamJob()
    job.name           = name
    job.workflow       = workflow
    job.application    = application
    job.description    = description
    job.args           = args
    job.num_nodes      = num_nodes
    job.ranks_per_node = ranks_per_node
    job.cpu_affinity   = cpu_affinity
    job.environ_vars   = environ_vars
    job.data           = {}
    job.save()
    
def submit(project='datascience',queue='debug-flat-quad',nodes=1,wall_minutes=30,job_mode='mpi',wf_filter=''):
    """
    Submits a job to the queue with the given parameters.
    Parameters
    ----------
    project: str, name of the project to be charged
    queue: str, queue name, can be: 'default', 'debug-cache-quad', or 'debug-flat-quad'
    nodes: int, Number of nodes, can be any integer from 1 to 4096.
    wall_minutes: int, max wall time in minutes
    job_mode: str, Balsam job mode, can be 'mpi', 'serial'
    wf_filter: str, Selects Balsam jobs that matches the given workflow filter.
    """
    from balsam import setup
    setup()
    from balsam.service import service
    from balsam.core import models
    QueuedLaunch = models.QueuedLaunch
    mylaunch = QueuedLaunch()
    mylaunch.project = project
    mylaunch.queue = queue
    mylaunch.nodes = nodes
    mylaunch.wall_minutes = wall_minutes
    mylaunch.job_mode = job_mode
    mylaunch.wf_filter = wf_filter
    mylaunch.prescheduled_only=False
    mylaunch.save()
    service.submit_qlaunch(mylaunch, verbose=True)

In [ ]:
# Adding apps to the database. No need to run twice.
add_app(name='trainer',
        executable='python /gpfs/mira-home/keceli/ffn/keceli_ffn/train_hvd.py',
        description='Distributed FFN training script',
        envscript='/lus/theta-fs0/projects/connectomics_aesp/balsam/training_env.sh')

add_app(name='inference',
        executable='python /gpfs/mira-home/keceli/ffn/keceli_ffn/run_inference.py',
        description='FFN inference script',
        envscript='/lus/theta-fs0/projects/connectomics_aesp/balsam/training_env.sh')

In [ ]:
# Adding a training job to the database
import time
TFRECORDFILE='/lus/theta-fs0/projects/datascience/keceli/run/f3n/training/tf_record_file'
GROUNDTRUTH='/lus/theta-fs0/projects/datascience/keceli/run/f3n/training/groundtruth.h5'
GRAYSCALE='/lus/theta-fs0/projects/datascience/keceli/run/f3n/training/grayscale_maps.h5'
BATCHSIZE=1
OPTIMIZER='adam'
TIMESTAMP=time.strftime("%y%m%d%H%M%S")
TRAINDIR=f'train_b{BATCHSIZE}_o{OPTIMIZER}_{TIMESTAMP}'

myargs = ''
myargs += f' --train_coords {TFRECORDFILE} '
myargs += f' --data_volumes valdation1:{GRAYSCALE}:raw '
myargs += f' --label_volumes valdation1:{GROUNDTRUTH}:stack '
myargs += f' --model_name convstack_3d.ConvStack3DFFNModel '
myargs += ''' --model_args "{\\"depth\\": 12, \\"fov_size\\": [33, 33, 33], \\"deltas\\": [8, 8, 8]}"'''
myargs += ' --image_mean 128 --image_stddev 33 '
myargs += ' --max_steps 400 --summary_rate_secs 60 ' 
myargs += f' --batch_size {BATCHSIZE} '
myargs += f' --optimizer {OPTIMIZER} '
myargs += ' --num_intra_threads 64 --num_inter_threads 1 '
myargs += f' --train_dir {TRAINDIR} '
print(myargs)

add_job(name='test_train',workflow='ffn_training',application='trainer',args=myargs)

In [76]:
# Generate subvolumes for inference 
import sys
sys.path.insert(0,'/gpfs/mira-home/keceli/ffn/keceli_ffn/')
sys.path.insert(0,'/lus/theta-fs0/projects/connectomics_aesp/keceli/pip_ffn/')
sys.path.insert(0,'/soft/datascience/tensorflow/tf1.13/')

from ffn.utils import bounding_box
from ffn.utils import geom_utils
bbox=bounding_box.BoundingBox(start=[0,0,0],size=[250,250,250])
def divide_bounding_box(bbox, subvolume_size, overlap):
    """
    Returns a list of bounding boxes that divides the given bounding box into subvolumes.
    Parameters
    ----------
    bbox: BoundingBox object,
    subvolume_size: list or tuple
    overlap: list or tuple
    """
    start = geom_utils.ToNumpy3Vector(bbox.start)
    size = geom_utils.ToNumpy3Vector(bbox.size)
    bbox = bounding_box.BoundingBox(start, size)
    calc = bounding_box.OrderlyOverlappingCalculator(outer_box=bbox, 
                                                    sub_box_size=subvolume_size, 
                                                    overlap=overlap, 
                                                    include_small_sub_boxes=True,
                                                    back_shift_small_sub_boxes=False)
    return [bb for bb in calc.generate_sub_boxes()]

In [77]:
# Add inference jobs for each subvolume
boxes = divide_bounding_box(bbox,subvolume_size=(100,100,100),overlap=(10,10,10))
print(f'Number of subvolumes: {len(boxes)}')
for i,box in enumerate(boxes):
    start = box.start
    size  = box.size
    inference_args  = ''' --inference_request="$(cat /lus/theta-fs0/projects/connectomics_aesp/keceli/inference/config.txt)" '''
    inference_args += f" --bounding_box 'start {{ x:{start[0]} y:{start[1]} z:{start[2]} }} size {{ x:{size[0]} y:{size[1]} z:{size[2]} }}' "
    add_job(name=f'debug_inference_{i}',
            workflow='debug_inference',
            application='inference',
            args=inference_args,
            environ_vars='OMP_NUM_THREADS=64')

Number of subvolumes: 27


In [80]:
# If you see 'Submit OK:', Job submission is succesful.
submit(project='connectomics_aesp',
       job_mode='serial',
       queue='debug-cache-quad',
       nodes=4,
       wall_minutes=59,
       wf_filter='debug_inference')

Submit OK: Qlaunch {   'command': '/lus/theta-fs0/projects/connectomics_aesp/balsam_database/qsubmit/qlaunch18.sh',
    'from_balsam': True,
    'id': 18,
    'job_mode': 'serial',
    'nodes': 4,
    'prescheduled_only': False,
    'project': 'connectomics_aesp',
    'queue': 'debug-cache-quad',
    'scheduler_id': 354724,
    'state': 'submitted',
    'wall_minutes': 59,
    'wf_filter': 'debug_inference'}


In [81]:
# If you see 'Submit OK:', Job submission is succesful.
submit(project='connectomics_aesp',
       queue='debug-flat-quad',
       nodes=3,
       wall_minutes=59,
       wf_filter='ffn_sub_inference')

Submit OK: Qlaunch {   'command': '/lus/theta-fs0/projects/connectomics_aesp/balsam_database/qsubmit/qlaunch19.sh',
    'from_balsam': True,
    'id': 19,
    'job_mode': 'mpi',
    'nodes': 3,
    'prescheduled_only': False,
    'project': 'connectomics_aesp',
    'queue': 'debug-flat-quad',
    'scheduler_id': 354725,
    'state': 'submitted',
    'wall_minutes': 59,
    'wf_filter': 'ffn_sub_inference'}


In [83]:
!pwd

/lus/theta-fs0/projects/datascience/jupyterhub
